In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('sales_train.csv')

df['date_lag1'] = df['date_block_num'] + 1

In [3]:
features = ["date_block_num","shop_id","item_id","item_price","item_cnt_day"]

df_sales = df[features]
# Concatenate the DataFrames along the columns (axis=1)
# result_df = pd.concat([existing_df, new_column], axis=1)

In [ ]:
# Gjør group by før vi gjør for loop.
# Gjør for loops på shop og item per hver unique måned
# Gjør om til Numpy array for at det skal gå raskere

In [4]:



# Step 1: Create a DataFrame with all possible combinations
# all_combinations = pd.DataFrame(
#     [(block, shop, item) for block in df_sales['date_block_num'].unique()
#      for shop in df_sales['shop_id'].unique()
#      for item in df_sales['item_id'].unique()],
#     columns=['date_block_num', 'shop_id', 'item_id']
# )



In [14]:
all_combinations_list = []

# Loop through each unique date_block_num
for block in df_sales['date_block_num'].unique():
    # Filter the DataFrame for the current date_block_num or earlier
    df_block = df_sales[df_sales['date_block_num'] <= block]
    
    # Generate combinations for shop_id and item_id
    combinations = [(block, shop, item) for shop in df_block['shop_id'].unique()
                    for item in df_block['item_id'].unique()]
    
    # Append combinations to the list
    all_combinations_list.extend(combinations)

# Create a DataFrame from the list of combinations
all_combinations = pd.DataFrame(all_combinations_list, columns=['date_block_num', 'shop_id', 'item_id'])# Step 2: Merge with existing sales data
merged_data = pd.merge(all_combinations, df_sales, on=['date_block_num', 'shop_id', 'item_id'], how='left')


In [ ]:
# Quality check: Display unique combinations for each date_block_num
for block in all_combinations['date_block_num'].unique():
    unique_combinations = all_combinations[all_combinations['date_block_num'] == block][['shop_id', 'item_id']].drop_duplicates()

In [15]:

# Step 3: Identify shops that didn't sell any products
shops_with_no_sales = merged_data[merged_data['item_cnt_day'].isnull()]['shop_id'].unique()


In [16]:

# Step 4: Fill missing values with 0
merged_data['item_cnt_day'].fillna(0, inplace=True)

merged_data['item_price'].fillna(0, inplace=True)

In [17]:
df_group=merged_data.groupby(['date_block_num', 'shop_id', 'item_id', 'item_price'])['item_cnt_day'].sum().reset_index()

In [18]:
# Save the result to a CSV file
result_file_path = 'merged_sales_data_grouped_new_method.csv'
df_group.to_csv(result_file_path, index=False)